In [1]:
import numpy as np
from sklearn.neural_network import MLPClassifier as mlp
from sklearn.metrics import cohen_kappa_score as kappa
from sklearn.model_selection import train_test_split

In [2]:
def populacao_inicial(tamanho_populacao, tamanho_cromossomo):
    return np.random.choice([0, 1], size=(tamanho_populacao,tamanho_cromossomo))

In [3]:
def crossover_two_best_n_random(populacao_atual,fitness_populacao,tamanho_cromossomo,n = 5):
    
    tamanho_populacao = len(populacao_atual)
    filhos = np.zeros((tamanho_populacao,tamanho_cromossomo))
    cont_filhos = 0
    for i in range(tamanho_populacao//2):
        
        tamanho_populacao_crossover = len(populacao_atual)#esse tamanho muda a cada iteracao
        
        if tamanho_populacao_crossover >= n: #escolhe os 5 somente se houverem pelo menos 5 na lista 
            escolhidos = np.random.choice(range(tamanho_populacao_crossover),size = n)#escolhe n pais aleatorios
        two_best = np.argsort(fitness_populacao)[-2:] #escolhe os 2 melhores entre os n
        pai = populacao_atual[two_best[0]]
        mae = populacao_atual[two_best[1]]
        
        populacao_atual = np.delete(populacao_atual,(two_best), axis=0) #remove os pais da populacao atual
        fitness_populacao = np.delete(fitness_populacao,(two_best)) #remove os fitness para ficar igual a lista de pais

        point_cross = np.random.choice(range(1,tamanho_cromossomo-1))
        filho1 = np.concatenate((pai[:point_cross],mae[point_cross:]))
        filho2 = np.concatenate((mae[:point_cross],pai[point_cross:]))
        
        filhos[cont_filhos,:] = filho1
        filhos[cont_filhos+1,:] = filho2
        cont_filhos += 2
        
    return filhos

In [4]:
def mutation(filhos, percentage=5):
    for id_f,f in enumerate(filhos):
        prob = np.random.choice(range(100))
        if prob > (100-percentage):
            #print('X-men')
            position = np.random.choice(range(0,tamanho_cromossomo))
            f[position] = int(not(f[position])) #troca 0->1 ou 1->0
            filhos[id_f] = f
    return filhos

In [5]:
def fitness(individuo):
    
    ## selecionar as features de acordo com o cromossomo: 1-> feature entra
    col = np.where(individuo==1)
    selected_features_train = features_train[:,col[0]]
    selected_features_validation = features_validation[:,col[0]]
    
    ## classificacao
    clf = mlp()
    clf.fit(selected_features_train,labels_train)
    pred = clf.predict(selected_features_validation)
    k = kappa(labels_validation,pred)
    
    return k

In [6]:
def next_generation(populacao_atual,fitness_populacao,filhos,fitness_filhos,tamanho_populacao):
    
    populacao = np.concatenate((populacao_atual,filhos))
    fitness = np.concatenate((fitness_populacao,fitness_filhos))
    next_generation = np.argsort(fitness)[-tamanho_populacao:] #escolhe os 'tamanho_populacao' melhores individuos
    populacao = populacao[next_generation]
    fitness = fitness[next_generation]
    
    return populacao,fitness

In [7]:
def condicao_parada(fitness_populacao):
    tamanho_populacao = len(fitness_populacao)
    
    if (np.sum(fitness_populacao) == tamanho_populacao): #condicao para saber se todos os individuos possuem 100% de acerto
        print('Parou com 100% de acerto')
        return False
    return True

## Ler CSV

In [39]:
file = '/Users/romuere/Downloads/CT-Scan_CNN_Otimizada.csv'
dados = np.loadtxt(file,delimiter=',',skiprows=1)
labels = dados[:,-1]
dados = dados[:,:-1]
# dividir dados: 70% para treino e validacao (onde 80% treino e 20% validacao) e 30% para teste
features_evolutivo, features_test, labels_evolutivo, labels_test = train_test_split(dados, labels, test_size=0.30)
features_train, features_validation, labels_train, labels_validation = train_test_split(features_evolutivo, labels_evolutivo, test_size=0.20)

In [40]:
dados.shape[1]

120

## Parâmetros do sistema evolutivo

In [41]:
tamanho_populacao = 10
tamanho_cromossomo = dados.shape[1]
quantidade_geracoes = 5
taxa_mutacao = 5

## Montando Evolutivo

In [42]:
populacao_atual = populacao_inicial(tamanho_populacao,tamanho_cromossomo)
fitness_populacao = [fitness(i) for i in populacao_atual]

In [46]:
qg = 0
while ((qg != quantidade_geracoes) and (condicao_parada(fitness_populacao))):
    filhos = crossover_two_best_n_random(populacao_atual,fitness_populacao,tamanho_cromossomo)
    filhos = mutation(filhos)
    fitness_filhos = [fitness(i) for i in filhos]
    populacao_atual,fitness_populacao = next_generation(populacao_atual,fitness_populacao,filhos,fitness_filhos,tamanho_populacao)
    print("Geracao: ",qg)
    qg += 1

Geracao:  0
Geracao:  1
Geracao:  2
Geracao:  3
Geracao:  4


## Buscar menor quantidade de Features 
- Em caso de dois ou mais indivíduos com a mesma fitness

In [47]:
quant_best_individuos = np.sum(fitness_populacao == fitness_populacao.max())
ordenado = np.argsort(fitness_populacao)[-quant_best_individuos:] #pega o indice do(s) melhor(es)

if quant_best_individuos > 1:
    #pegar o individuo com menos atributos
    tam_best = tamanho_cromossomo
    for i in ordenado:
        if np.sum(populacao_atual[i]) <= tam_best:
            best = i
            tam_best = np.sum(populacao_atual[i])
else:
    best = ordenado[0]
    
individuo_final = populacao_atual[best]
print('Number of features: ',np.sum(individuo_final))

Number of features:  57.0


In [50]:
individuo_final

array([1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       1.])

## Ajustar vetores para selecionar os atributos

In [48]:
def evaluate(individuo):
    
    ## selecionar as features de acordo com o cromossomo: 1-> feature entra
    col = np.where(individuo==1)
    selected_features_evolutivo = features_evolutivo[:,col[0]]
    selected_features_test = features_test[:,col[0]]
    
    ## classificacao
    clf = mlp()
    clf.fit(selected_features_evolutivo,labels_evolutivo)
    pred = clf.predict(selected_features_test)
    k = kappa(labels_test,pred)
    
    return k

In [49]:
evaluate(individuo_final)

0.9785096670440696